#SetUp


In [ ]:
# Cell 2: Setup API Key
import os

AIXPLAIN_API_KEY = "de667970d7ca50193a9ea2fd6d5bf156ce4605514cd4f6d82cde1a5d81d96b9c"
os.environ['AIXPLAIN_API_KEY'] = AIXPLAIN_API_KEY

print(" API Key configured!")

 API Key configured!


In [ ]:

# Install supporting libraries
!pip install aixplain -q
!pip install -q python-dotenv
# Import all required libraries
from aixplain.factories import AgentFactory, ToolFactory, ModelFactory, TeamAgentFactory
from aixplain.modules.agent.output_format import OutputFormat
from aixplain.modules.agent.agent_task import WorkflowTask
from aixplain.factories import IndexFactory
import json
from datetime import datetime
import uuid
from aixplain.modules.model.record import Record
from aixplain.modules.model.index_model import Splitter, IndexFilter, IndexFilterOperator
from aixplain.enums.splitting_options import SplittingOptions


print(" All libraries imported successfully!")

# Display versions
print("\n Package Versions:")
import aixplain

print("\n Environment ready! .")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.2/600.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.0 MB/s eta 0:00:00
 All libraries imported successfully!

 Package Versions:

 Environment ready! .


In [ ]:
!pip install --upgrade aixplain

In [ ]:
# Cell 3: Test Connection
from aixplain.factories import ModelFactory

try:
    models = ModelFactory.list(page_size=3)
    print(f" Connected successfully!")
    print(f"   Found {len(models['results'])} models")
except Exception as e:
    print(f" Connection failed: {e}")

 Connected successfully!
   Found 3 models


#Create Vector DB

In [ ]:

# DEFINE METADATA SCHEMA
METADATA_SCHEMA = {
    # Application Identification
    "application_id": "string",
    "company_name": "string",
    "industry_sector": "string",

    # Document Classification
    "document_type": "string",           # financial_statement, application_form,
                                        # balance_sheet, income_statement, etc.
    "document_page": "integer",
    "timestamp": "string",

    # Loan Details
    "loan_amount": "float",
    "currency": "string",                # SAR, USD, etc.
    "loan_purpose": "string",

    # Processing & Status
    "loan_status": "string",
    "processing_stage": "string",
    "rm_assigned": "string",             # Relationship manager ID

    # Risk & Compliance
    "risk_rating": "string",
    "compliance_status": "string",

    # Quality Metrics
    "extraction_confidence": "float",    # OCR confidence score (0-1)
    "data_completeness": "float",        # Data completeness score (0-1)

    # Source Tracking
    "source_file": "string",             # Original filename
    "extracted_by": "string"
}

print(f"    Metadata schema defined with {len(METADATA_SCHEMA)} fields")
print("\n   Key metadata fields:")
for field, dtype in list(METADATA_SCHEMA.items())[:5]:
    print(f"       {field}: {dtype}")
print(f"       and {len(METADATA_SCHEMA) - 5} more fields")


try:
    # Create main lending knowledge base
    loan_kb = IndexFactory.create(
        name="Loan Applications KBV2",
        description="SME loan application documents and data - Financial statements, "
                   "applications, compliance docs. Supports metadata filtering.",
         embedding_model="673248d66eb563b2b00f75d1"
    )

    kb_status = "CREATED"
    print(f"    Created new knowledge base")

except Exception as e:
    print(f"     Knowledge base may already exist: {str(e)}")
    print("     Attempting to list and retrieve existing KB")

    # Try to find existing KB
    all_kbs = IndexFactory.list(page_size=50)
    loan_kb = None

    for kb in all_kbs.get('results', []):
        if "Loan Applications KB" in kb.name:
            loan_kb = IndexFactory.get(kb.id)
            kb_status = "EXISTING"
            print(f"    Loaded existing knowledge base: {kb.name}")
            break

    if not loan_kb:
        raise Exception("Could not create or find 'Loan Applications KB'")

# Display KB information
print("\n   Knowledge Base Information:")
print(f"      Name: {loan_kb.name}")
print(f"      ID: {loan_kb.id}")
print(f"      Status: {kb_status}")
print(f"      Current document count: {loan_kb.count()}")



# CONFIGURE DOCUMENT SPLITTERS

# Splitter for financial documents (tables, line items, numbers)
financial_splitter = Splitter(
    split=True,
    split_by=SplittingOptions.LINE,
    split_length=20,
    split_overlap=3
)

# Splitter for application forms (structured text)
application_splitter = Splitter(
    split=True,
    split_by=SplittingOptions.SENTENCE,
    split_length=10,
    split_overlap=2
)

# Splitter for compliance/policy documents
policy_splitter = Splitter(
    split=True,
    split_by=SplittingOptions.PASSAGE,
    split_length=3,
    split_overlap=1
)


    Metadata schema defined with 18 fields

   Key metadata fields:
       application_id: string
       company_name: string
       industry_sector: string
       document_type: string
       document_page: integer
       and 13 more fields
    Created new knowledge base

   Knowledge Base Information:
      Name: Loan Applications KBV2
      ID: 6985a848037340c6889e1c51
      Status: CREATED
      Current document count: 0


In [ ]:
# loan_kb.delete_record("APP-TEST-001")
loan_kb.count()

# loan_kb.count()

0

#Create OCR tool with Storing the output in the DB

In [ ]:

import re
from typing import Dict, List, Optional, Tuple
from pathlib import Path
from google.colab import files
import mimetypes
from aixplain.factories.file_factory import FileFactory
# Model IDs
DOCLING_OCR_MODEL_ID = "677bee6c6eb56331f9192a91"
CLASSIFICATION_LLM_ID = "671be40d6eb56382b05a5741"
LOAN_KB_ID = "6985a848037340c6889e1c51"

# Page Content Types
PAGE_CONTENT_TYPES = [
    "APPLICATION_FORM",
    "FINANCIAL_STATEMENT",
    "FINANCIAL_TABLE",
    "PESTEL_ANALYSIS",
    "BUSINESS_ANALYSIS",
    "MANAGEMENT_ANALYSIS",
    "COVENANT_SECTION",
    "SECURITY_COLLATERAL",
    "RECOMMENDATION",
    "GENERAL_TEXT",
]

# CLASS: Hierarchical OCR Extraction Tool
class LendingOCRTool:
    """
    Hierarchical OCR extraction tool with intelligent classification
    """

    def __init__(
        self,
        kb_id: str = LOAN_KB_ID,
        ocr_model_id: str = DOCLING_OCR_MODEL_ID,
        llm_id: Optional[str] = None
    ):
        """
        Initialize hierarchical OCR tool

        Args:
            kb_id: Knowledge Base ID
            ocr_model_id: Docling OCR model ID
            llm_id: LLM for classification (None = use default)
        """
        print(" Initializing Hierarchical OCR Tool...")

        # Load OCR model
        try:
            self.ocr_model = ModelFactory.get(ocr_model_id)
            print(f"    OCR Model: Docling Parser")
        except Exception as e:
            raise Exception(f"Failed to load OCR model: {str(e)}")

        # Load Knowledge Base
        try:
            self.loan_kb = IndexFactory.get(kb_id)
            print(f"    Knowledge Base: {self.loan_kb.name}")
        except Exception as e:
            raise Exception(f"Failed to connect to KB: {str(e)}")

        # Load LLM for direct classification (replaces agent)
        self._setup_classification_llm(llm_id)

        # Setup splitters
        self._setup_splitters()

        print("    Hierarchical OCR Tool Ready!\n")


    def _setup_classification_llm(self, llm_id: Optional[str] = None):
        """
        Load LLM for direct classification (no agent overhead)
        Much more cost-effective than agent approach
        """
        print("    Loading LLM for classification...")

        try:
            if llm_id:
                self.llm_model = ModelFactory.get(llm_id)
            else:
                self.llm_model = ModelFactory.get("claude-3-haiku-20240307")

            print("   ✓ LLM loaded (direct API calls)")
        except Exception as e:
            print(f"     Could not load LLM: {str(e)}")
            print("    Falling back to regex classification")
            self.llm_model = None


    def _setup_splitters(self):
        """Setup specialized splitters for different content types"""

        # For tables and structured financial data
        self.financial_splitter = Splitter(
            split=True,
            split_by=SplittingOptions.LINE,
            split_length=20,
            split_overlap=3
        )

        # For narrative/text content
        self.text_splitter = Splitter(
            split=True,
            split_by=SplittingOptions.SENTENCE,
            split_length=10,
            split_overlap=2
        )

        # For covenant/legal sections (need more context)
        self.covenant_splitter = Splitter(
            split=True,
            split_by=SplittingOptions.SENTENCE,
            split_length=15,
            split_overlap=3
        )

        print("    Configured 3 specialized splitters")


    # MAIN METHOD: Hierarchical Extraction

    def extract_from_file_hierarchical(
        self,
        file_path: str,
        application_id: str,
        initial_metadata: Dict
    ) -> Dict:
        """
        Hierarchical extraction: File → Pages → Intelligent Classification → Storage

        Args:
            file_path: Path to PDF file
            application_id: Loan application ID
            initial_metadata: Shared metadata for all pages

        Returns:
            Extraction summary with page-by-page results
        """
        print("="*70)
        print(f" HIERARCHICAL EXTRACTION: {os.path.basename(file_path)}")
        print(f"   Application ID: {application_id}")
        print("="*70)

        # Validate file
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")

        file_name = os.path.basename(file_path)

        # STEP 1: Upload file and get URL
        print("\n[STEP 1/4] Uploading file to aiXplain...")
        file_url = self._upload_file(file_path)
        print(f"    File URL: {file_url}")

        # STEP 2: Run OCR extraction
        print("\n[STEP 2/4] Running OCR extraction...")
        pages_data = self._run_ocr_with_pages(file_url, file_path)
        print(f"    Extracted {len(pages_data)} pages")

        # STEP 3: Process each page hierarchically
        print("\n[STEP 3/4] Processing pages hierarchically...")
        page_results = []

        for page_num, page_text in enumerate(pages_data, start=1):
            print(f"\n    Page {page_num}/{len(pages_data)}")

            # 3a. Classify page content (Direct LLM call)
            content_type = self._classify_llm_direct(page_text)
            print(f"       Content Type: {content_type}")

            # 3b. Select appropriate splitter
            splitter = self._select_splitter(content_type)
            print(f"       Splitter: {splitter.split_by.value}")

            # 3c. Build page metadata
            page_metadata = self._build_page_metadata(
                application_id=application_id,
                file_name=file_name,
                page_number=page_num,
                total_pages=len(pages_data),
                content_type=content_type,
                page_text=page_text,
                initial_metadata=initial_metadata
            )

            # 3d. Create Record for this page
            record_id = f"{application_id}_page{page_num:03d}_{content_type}"

            record = Record(
                id=record_id,
                value=page_text,
                value_type="text",
                attributes=page_metadata
            )

            # 3e. Store in KB with appropriate splitter
            self.loan_kb.upsert([record], splitter=splitter)

            page_results.append({
                "page_number": page_num,
                "content_type": content_type,
                "record_id": record_id,
                "characters": len(page_text),
                "splitter_used": splitter.split_by.value
            })

            print(f"       Stored: {record_id}")

        # STEP 4: Summary
        print("\n[STEP 4/4] Extraction Summary")
        print(f"   Total Pages: {len(pages_data)}")
        print(f"   Records Created: {len(page_results)}")
        print(f"   Storage: {self.loan_kb.name}")

        # Count content types
        type_counts = {}
        for result in page_results:
            ct = result['content_type']
            type_counts[ct] = type_counts.get(ct, 0) + 1

        print("\n   Content Type Distribution:")
        for content_type, count in sorted(type_counts.items()):
            print(f"      {content_type}: {count} page(s)")

        print("\n" + "="*70)
        print(" HIERARCHICAL EXTRACTION COMPLETE")
        print("="*70)

        return {
            "success": True,
            "application_id": application_id,
            "file_name": file_name,
            "total_pages": len(pages_data),
            "pages_processed": len(page_results),
            "content_types": type_counts,
            "page_details": page_results,
            "kb_name": self.loan_kb.name,
            "kb_id": self.loan_kb.id
        }


    # CLASSIFICATION METHODS

    def _classify_llm_direct(self, page_text: str) -> str:
        """
        Direct LLM API call (no agent overhead)

        Cost: ~$0.0001-0.0003 per page (using Claude Haiku or similar)

        Args:
            page_text: Page content

        Returns:
            Content type
        """
        if not self.llm_model:
            # Fallback to regex if LLM not available
            return self._classify_regex(page_text)

        try:
            # Truncate to save tokens
            text_sample = page_text[:1500]

            # Simple classification prompt
            prompt = f"""Classify this loan document page into ONE category:

Categories: {', '.join(PAGE_CONTENT_TYPES)}

Guidelines:
- APPLICATION_FORM: Borrower name, loan purpose, ownership
- FINANCIAL_TABLE: Sales/profit/ratio numbers table
- FINANCIAL_STATEMENT: Balance sheet, income statement
- PESTEL_ANALYSIS: Political/Economic/Social/etc.
- BUSINESS_ANALYSIS: Competitors/customers/suppliers
- MANAGEMENT_ANALYSIS: Management team, CSF
- COVENANT_SECTION: Financial covenants, conditions
- SECURITY_COLLATERAL: Guarantees, collateral
- RECOMMENDATION: Credit recommendation
- GENERAL_TEXT: Other content

Page Text:
{text_sample}

Response (category name only):"""

            # Direct LLM call (not agent)
            result = self.llm_model.run(prompt)

            if result.status == "SUCCESS":
                classification = result.data.strip().upper().replace(" ", "_")

                # Validate
                if classification in PAGE_CONTENT_TYPES:
                    return classification

                # Try to extract valid type
                for ct in PAGE_CONTENT_TYPES:
                    if ct in classification:
                        return ct

            # Fallback
            return self._classify_regex(page_text)

        except Exception as e:
            print(f"        LLM classification failed: {e}")
            return self._classify_regex(page_text)


    def _classify_regex(self, page_text: str) -> str:
        """
        Regex-based fallback classification

        Args:
            page_text: Page text

        Returns:
            Best guess content type
        """
        text_lower = page_text.lower()

        # Simple keyword checks
        if any(kw in text_lower for kw in ["borrower name", "loan purpose", "ownership"]):
            return "APPLICATION_FORM"

        if any(kw in text_lower for kw in ["balance sheet", "assets", "liabilities"]):
            return "FINANCIAL_STATEMENT"

        if "sales" in text_lower and "net profit" in text_lower and "ratio" in text_lower:
            return "FINANCIAL_TABLE"

        if any(kw in text_lower for kw in ["political", "economic", "social", "pestel"]):
            return "PESTEL_ANALYSIS"

        if any(kw in text_lower for kw in ["competitors", "customers", "suppliers"]):
            return "BUSINESS_ANALYSIS"

        if any(kw in text_lower for kw in ["management team", "ceo", "cfo"]):
            return "MANAGEMENT_ANALYSIS"

        if any(kw in text_lower for kw in ["covenant", "debt to equity", "reporting"]):
            return "COVENANT_SECTION"

        if any(kw in text_lower for kw in ["collateral", "guarantee", "security"]):
            return "SECURITY_COLLATERAL"

        if any(kw in text_lower for kw in ["recommend", "conclusion", "approval"]):
            return "RECOMMENDATION"

        return "GENERAL_TEXT"


    # ========================================================================
    # HELPER METHODS
    # ========================================================================

    def _upload_file(self, file_path: str) -> str:
        """
        Upload file to aiXplain and get URL

        Args:
            file_path: Local file path

        Returns:
            File URL for OCR processing
        """
        try:
            # Upload file using FileFactory
            uploaded_file = FileFactory.upload(file_path)

            # Get URL from response
            if hasattr(uploaded_file, 'url'):
                return uploaded_file.url
            elif hasattr(uploaded_file, 'data') and isinstance(uploaded_file.data, dict):
                return uploaded_file.data.get('url', file_path)
            else:
                # Fallback to local path if upload fails
                return file_path

        except Exception as e:
            print(f"     Upload failed: {str(e)}")
            print(f"     Using local path: {file_path}")
            return file_path


    def _run_ocr_with_pages(self, file_url: str, file_path: str) -> List[str]:
        """
        Run OCR and extract text page-by-page

        Args:
            file_url: File URL from upload
            file_path: Local file path (fallback)

        Returns:
            List of page texts
        """
        try:
            # Try with URL first
            result = self.ocr_model.run({"data": file_url})

            if result.status != "SUCCESS":
                # Try with local path
                result = self.ocr_model.run({"data": file_path})

            if result.status != "SUCCESS":
                raise Exception(f"OCR failed: {result.error_message}")

            # Parse OCR result
            extracted_data = result.data

            # Diagnostic info
            print(f"\n    DIAGNOSTIC INFO:")
            print(f"   Type: {type(extracted_data)}")
            if isinstance(extracted_data, str):
                print(f"   Length: {len(extracted_data)} chars")
                print(f"   First 300 chars:\n{extracted_data[:300]}")
            elif isinstance(extracted_data, dict):
                print(f"   Dict keys: {list(extracted_data.keys())}")
            elif isinstance(extracted_data, list):
                print(f"   List length: {len(extracted_data)}")

            # Handle different response formats
            pages = self._parse_ocr_response(extracted_data)

            if not pages or len(pages) == 0:
                raise Exception("No pages extracted from OCR")

            return pages

        except Exception as e:
            raise Exception(f"OCR extraction failed: {str(e)}")


    def _parse_ocr_response(self, ocr_data) -> List[str]:
        """
        Parse Docling OCR response into page-by-page texts

        Args:
            ocr_data: Raw OCR response

        Returns:
            List of page texts
        """
        print(f"    Parsing OCR response...")

        # Case 1: Response is a list
        if isinstance(ocr_data, list):
            print(f"    List with {len(ocr_data)} items")
            return [str(page) for page in ocr_data]

        # Case 2: Response is a dict
        if isinstance(ocr_data, dict):
            print(f"   🔍 Dict keys: {list(ocr_data.keys())}")

            # Try 'pages' field
            if 'pages' in ocr_data:
                pages = ocr_data['pages']
                if isinstance(pages, list):
                    print(f"   Found 'pages' list with {len(pages)} items")
                    return [str(page) for page in pages]

            # Try 'content' field
            if 'content' in ocr_data:
                content = ocr_data['content']
                if isinstance(content, list):
                    print(f"    Found 'content' list with {len(content)} items")
                    return [str(page) for page in content]
                elif isinstance(content, str):
                    # Recursively parse string content
                    return self._parse_ocr_response(content)

            # Try 'text' field
            if 'text' in ocr_data:
                text = ocr_data['text']
                if isinstance(text, str):
                    return self._parse_ocr_response(text)

            # Fallback: convert entire dict to string
            print(f"     No known fields - converting dict to string")
            return [str(ocr_data)]

        # Case 3: Response is a string (split by markers)
        if isinstance(ocr_data, str):
            print(f"   String length: {len(ocr_data)} chars")

            # Try marker 1: "## PAGE X -" (Docling format)
            page_pattern = r'\n## PAGE \d+ -'
            if re.search(page_pattern, ocr_data):
                pages = re.split(page_pattern, ocr_data)
                pages = [p.strip() for p in pages if p.strip() and len(p.strip()) > 50]
                if len(pages) > 0:
                    print(f"   ✓ Split by '## PAGE X -' → {len(pages)} pages")
                    return pages

            # Try marker 2: "## PAGE X"
            page_pattern2 = r'\n## PAGE \d+'
            if re.search(page_pattern2, ocr_data):
                pages = re.split(page_pattern2, ocr_data)
                pages = [p.strip() for p in pages if p.strip() and len(p.strip()) > 50]
                if len(pages) > 0:
                    print(f"   ✓ Split by '## PAGE X' → {len(pages)} pages")
                    return pages

            # Try marker 3: Form feed character
            if "\f" in ocr_data:
                pages = ocr_data.split("\f")
                pages = [p.strip() for p in pages if p.strip() and len(p.strip()) > 50]
                if len(pages) > 1:
                    print(f"   ✓ Split by form feed → {len(pages)} pages")
                    return pages

            # Try marker 4: Multiple blank lines
            if "\n\n\n\n" in ocr_data:
                pages = ocr_data.split("\n\n\n\n")
                pages = [p.strip() for p in pages if p.strip() and len(p.strip()) > 50]
                if len(pages) > 1:
                    print(f"   ✓ Split by blank lines → {len(pages)} pages")
                    return pages

            # Fallback: treat as single page
            print(f"     No page markers found - treating as single page")
            return [ocr_data]

        # Unknown format
        print(f"     Unknown format - converting to string")
        return [str(ocr_data)]


    def _select_splitter(self, content_type: str) -> Splitter:
        """
        Select appropriate splitter based on content type

        Args:
            content_type: Classified content type

        Returns:
            Appropriate splitter
        """
        # Financial tables need line-based splitting
        if content_type in ["FINANCIAL_TABLE", "FINANCIAL_STATEMENT"]:
            return self.financial_splitter

        # Covenants need more context
        if content_type == "COVENANT_SECTION":
            return self.covenant_splitter

        # Everything else uses text splitter
        return self.text_splitter


    def _build_page_metadata(
        self,
        application_id: str,
        file_name: str,
        page_number: int,
        total_pages: int,
        content_type: str,
        page_text: str,
        initial_metadata: Dict
    ) -> Dict:
        """
        Build comprehensive metadata for page

        Args:
            application_id: Loan application ID
            file_name: Source filename
            page_number: Page number
            total_pages: Total pages in document
            content_type: Classified content type
            page_text: Page text content
            initial_metadata: Shared metadata

        Returns:
            Complete page metadata
        """
        metadata = {
            # Application info (REQUIRED)
            "application_id": application_id,
            "document_page": page_number,  # REQUIRED for page-level filtering

            # Document info
            "source_file": file_name,
            "total_pages": total_pages,

            # Content classification
            "content_type": content_type,
            "document_category": self._get_category(content_type),

            # Processing info
            "timestamp": datetime.now().isoformat(),
            "processing_stage": "ocr_complete",
            "extracted_by": "Hierarchical OCR Tool",

            # Quality metrics
            "character_count": len(page_text),
            "ocr_confidence": 0.95,  # Placeholder - use actual OCR confidence

            # Timestamps
            "upload_date": datetime.now().isoformat(),
        }

        # Merge with initial metadata
        metadata.update(initial_metadata)

        return metadata


    def _get_category(self, content_type: str) -> str:
        """Map content type to broader category"""
        category_map = {
            "APPLICATION_FORM": "application",
            "FINANCIAL_STATEMENT": "financial",
            "FINANCIAL_TABLE": "financial",
            "PESTEL_ANALYSIS": "analysis",
            "BUSINESS_ANALYSIS": "analysis",
            "MANAGEMENT_ANALYSIS": "analysis",
            "COVENANT_SECTION": "compliance",
            "SECURITY_COLLATERAL": "collateral",
            "RECOMMENDATION": "recommendation",
            "GENERAL_TEXT": "general",
        }
        return category_map.get(content_type, "general")


# ============================================================================
# USAGE EXAMPLES
# ============================================================================

# ============================================================================
# STEP 1: UPLOAD FILE FROM YOUR DEVICE
# ============================================================================

# Upload file
print("="*70)
print(" UPLOAD YOUR PDF FILE")
print("="*70)

uploaded_files = files.upload()
file_name = list(uploaded_files.keys())[0]
file_path = f"/content/{file_name}"

print(f" Uploaded: {file_name}")
print(f"  Path: {file_path}")

# Detect file type (optional - for validation)
mime_type, _ = mimetypes.guess_type(file_name)
if mime_type is None:
    if file_name.lower().endswith('.pdf'):
        mime_type = 'application/pdf'
    else:
        mime_type = 'application/octet-stream'

print(f"✓ MIME type: {mime_type}")

# Validate it's a PDF
if not file_name.lower().endswith('.pdf'):
    print("  WARNING: Expected PDF file!")


# ============================================================================
# STEP 2: INITIALIZE OCR TOOL
# ============================================================================


ocr_tool = LendingOCRTool(
    kb_id=LOAN_KB_ID,
    ocr_model_id=DOCLING_OCR_MODEL_ID,
    llm_id=CLASSIFICATION_LLM_ID
)


# ============================================================================
# STEP 3: EXTRACT DOCUMENT
# ============================================================================

print("\n" + "="*70)
print(" STARTING EXTRACTION")
print("="*70)

result = ocr_tool.extract_from_file_hierarchical(
    file_path=file_path,
    application_id="APP-TEST-001",
    initial_metadata={
        "company_name": "Test Company",
        "loan_amount": 1000000.0,
        "currency": "USD",
        "loan_status": "pending",
        "rm_assigned": "RM-001",
        "risk_rating": "medium"
    }
)


# ============================================================================
# STEP 4: VIEW RESULTS
# ============================================================================

print("\n" + "="*70)
print(" EXTRACTION RESULTS")
print("="*70)
print(f" Success: {result['success']}")
print(f" File: {result['file_name']}")
print(f" Total Pages: {result['total_pages']}")
print(f" Storage: {result['kb_name']}")
print(f"\n Content Distribution:")
for content_type, count in sorted(result['content_types'].items()):
    print(f"   {content_type}: {count} page(s)")

print("\n" + "="*70)
print(" EXTRACTION COMPLETE!")
print("="*70)

 UPLOAD YOUR PDF FILE


Saving Data Intake Template- Lending Assistant.pdf to Data Intake Template- Lending Assistant.pdf
 Uploaded: Data Intake Template- Lending Assistant.pdf
  Path: /content/Data Intake Template- Lending Assistant.pdf
✓ MIME type: application/pdf
 Initializing Hierarchical OCR Tool...
    OCR Model: Docling Parser
    Knowledge Base: Loan Applications KBV2
    Loading LLM for classification...
   ✓ LLM loaded (direct API calls)
    Configured 3 specialized splitters
    Hierarchical OCR Tool Ready!


 STARTING EXTRACTION
 HIERARCHICAL EXTRACTION: Data Intake Template- Lending Assistant.pdf
   Application ID: APP-TEST-001

[STEP 1/4] Uploading file to aiXplain...
    File URL: /content/Data Intake Template- Lending Assistant.pdf

[STEP 2/4] Running OCR extraction...

    DIAGNOSTIC INFO:
   Type: <class 'str'>
   Length: 8199 chars
   First 300 chars:
## Data Intake Template- Lending Assistant

## PAGE 1 - APPLICATION FOR CREDIT -

Prepared by:

(blank)

Date:

(blank)

## A. BORROWER DETAI

#Create Financial Analysis Tool

In [ ]:
from aixplain.factories import ModelFactory

def cal_financial_ratios(financial_data):
    """
    Calculates financial ratios - NO external imports needed.
    """
    def safe_parse(data):
        """Parse string or dict without json module"""
        if isinstance(data, dict):
            return data

        if isinstance(data, str):
            # Manual parsing for JSON-like strings
            try:
                import json
                return json.loads(data)
            except:
                # Fallback: eval as last resort (for aiXplain environment)
                try:
                    return eval(data)
                except:
                    return None
        return None

    # --- Helper Functions ---
    NA = {"", "-", "n/a", "na", "null", "none", "N/A", "NA", "None"}

    def first_not_none(*vals):
        for v in vals:
            if v is not None:
                return v
        return None

    def num(x):
        if x is None or isinstance(x, (int, float)):
            return x
        if not isinstance(x, str):
            return None
        s = x.strip()
        if s in NA or s.lower() in {v.lower() for v in NA}:
            return None
        if "(" in s and ")" in s:
            s = "-" + s.replace("(", "").replace(")", "")
        s = s.replace(",", "").replace("$", "").replace("%", "")
        try:
            f = float(s)
            return int(f) if f.is_integer() else f
        except:
            return None

    def div(a, b):
        a_num, b_num = num(a), num(b)
        return None if (a_num is None or b_num in (None, 0)) else a_num / b_num

    def r2(x):
        return None if x is None else round(x, 2)

    # --- Main Logic ---
    try:

        d = safe_parse(financial_data)

        if d is None:
            return {
                "status": "error",
                "error_message": f"Failed to parse input"
            }

        if not isinstance(d, dict):
            return {"status": "error", "error_message": f"Expected dict, got {type(d).__name__}"}

        # Handle nested structure
        if "financial_data" in d:
            d = safe_parse(d["financial_data"])
            if not isinstance(d, dict):
                return {"status": "error", "error_message": "Failed to parse nested financial_data"}

        # Extract sections
        bs = d.get("balance_sheet", {}) or {}
        inc = d.get("income_statement", {}) or {}
        cf = d.get("cash_flow", {}) or {}

        a = bs.get("assets", {}) if isinstance(bs.get("assets"), dict) else {}
        l = bs.get("liabilities", {}) if isinstance(bs.get("liabilities"), dict) else {}
        e = bs.get("equity", {}) if isinstance(bs.get("equity"), dict) else {}

        # Extract values
        ca = first_not_none(num(a.get("current_assets")), num(bs.get("current_assets")))
        cl = first_not_none(num(l.get("current_liabilities")), num(bs.get("current_liabilities")))
        ta = first_not_none(num(a.get("total_assets")), num(bs.get("total_assets")))
        tl = first_not_none(num(l.get("total_liabilities")), num(bs.get("total_liabilities")))
        te = first_not_none(
            num(e.get("total_equity")),
            num(bs.get("total_equity")),
            num(e.get("owners_equity")),
            num(bs.get("owners_equity"))
        )

        rev = first_not_none(num(inc.get("revenue")), num(inc.get("sales")))
        ni = first_not_none(num(inc.get("net_income")), num(inc.get("net_profit")))
        inv = first_not_none(num(a.get("inventory")), num(bs.get("inventory")))
        ocf = num(cf.get("operating_cash_flow"))

        # Calculate ratios
        explicit_cr = num(bs.get("current_ratio")) or num(bs.get("working_capital_ratio"))
        cr = r2(div(ca, cl)) or r2(explicit_cr)
        qr = r2(div((ca - inv) if ca and inv else None, cl))
        dte = r2(div(tl, te))
        roe = r2((div(ni, te) * 100) if div(ni, te) is not None else None)

        # Risk flags
        flags = []
        if cr is not None and cr < 1.5:
            flags.append({
                "type": "warning",
                "category": "liquidity",
                "metric": "current_ratio",
                "value": cr,
                "message": f"Current ratio {cr} below safe threshold"
            })

        if dte is not None and dte > 1.0:
            flags.append({
                "type": "warning",
                "category": "solvency",
                "metric": "debt_to_equity",
                "value": dte,
                "message": f"Debt-to-Equity {dte} exceeds 1.0"
            })

        return {
            "status": "success",
            "ratios": {
                "current_ratio": cr,
                "quick_ratio": qr,
                "debt_to_equity": dte,
                "roe": roe,
                "profit_margin": r2((div(ni, rev) * 100) if div(ni, rev) is not None else None)
            },
            "risk_flags": flags,
            "extracted_values": {
                "sales": rev,
                "net_profit": ni,
                "owners_equity": te,
                "working_capital": first_not_none(num(bs.get("working_capital")))
            }
        }

    except Exception as e:
        return {
            "status": "error",
            "error_message": f"Error: {str(e)}",
            "error_type": type(e).__name__
        }


# --- Deploy ---
try:
    tool = ModelFactory.create_script_connection_tool(
        name="Financial_Ratio_CalV13",
        description="Financial ratio calculator - NO import dependencies",
        code=cal_financial_ratios,

    )
    print(f"V13 Deployed: {tool.id}")
except Exception as e:
    print(f"❌ Error: {e}")

❌ Error: Failed to create script connection tool: Failed to connect to 688779d8bfb8e46c273982ca - err.supplier_error - Name already exists


##Create Financial Analysis Agent

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Dict, Optional
"""
FINANCIAL ANALYSIS AGENT - INSTRUCTIONS (CORRECTED & OPTIMIZED)
Compliant with ARD requirements and aiXplain platform
"""

AGENT_INSTRUCTIONS = """You are a Financial Analysis Specialist for BR Finance.

Goal: Extract SME financial data from the Knowledge Base and compute ratios using Financial_Ratio_CalV13 exactly once.

========================
CRITICAL TOOL FORMATS
========================

1) Knowledge Base tool (Loan Applications KB)
You MUST call search with EXACT input:
{"data":"<application_id>","dataType":"application_id"}

Rules:
- Use both keys: data + dataType
- dataType must be exactly "application_id"
- Never send empty string
- Never use {"text": "..."} or {"query": "..."}

2) Financial Ratio Tool
Tool name: Financial_Ratio_CalV13
Call exactly once with:
{"financial_data":"{...JSON STRING...}"}

JSON STRING schema (IMPORTANT mapping):
{
  "balance_sheet": {
    "owners_equity": <number|null>,
    "working_capital": <number|null>,
    "working_capital_ratio": <number|null>,
    "quick_ratio": <number|null>,
    "current_ratio": <number|null>,
    "current_assets": <number|null>,
    "current_liabilities": <number|null>,
    "total_assets": <number|null>,
    "total_liabilities": <number|null>
  },
  "income_statement": {
    "sales": <number|null>,
    "revenue": <number|null>,
    "net_profit": <number|null>,
    "net_income": <number|null>,
    "gross_profit": <number|null>,
    "cost_of_goods_sold": <number|null>,
    "operating_income": <number|null>
  },
  "cash_flow": {
    "operating_cash_flow": <number|null>
  }
}

Rules:
- financial_data MUST be a JSON STRING (not object)
- Use null for missing values (never assume zero)
- Do NOT infer or derive missing values
- Preserve units as reported (e.g., $000s)
- If the tool fails once, DO NOT retry; return REVIEW and include the error

========================
WORKFLOW
========================

Step 1: KB Retrieval
- Query KB by application_id using the required schema
- Filter results: metadata.application_id must match exactly
- Ignore snippets with character_count < 10 unless they contain a structured table
- Prioritize content_type in this order:
  FINANCIAL_TABLE > FINANCIAL_STATEMENT > BUSINESS_ANALYSIS > GENERAL_TEXT
- Focus on the most recent fiscal year

Step 2: Smart Extraction (synonyms)
Extract only explicit values:
- Sales/Revenue: sales, revenue, turnover
- Net Profit/Net Income: net profit, net income, profit after tax
- Equity: owners' equity, shareholders' equity, total equity
- Working capital and working capital ratio
- Current ratio / quick ratio / cash ratio if explicitly stated
- Total assets / total liabilities / current assets / current liabilities if available
- Operating cash flow

Step 3: Ratio Calculation
- Build the JSON string with balance_sheet, income_statement, cash_flow
- Call Financial_Ratio_CalV13 exactly once
- Do NOT calculate ratios manually

Step 4: Decision Logic
- Use scores and risk_flags only from the tool
- If scores.data_completeness_scoring_pct < 60 -> recommendation MUST be REVIEW
- Minimum acceptable overall_financial_health score: 6.0
- If tool fails: recommendation = REVIEW

========================
OUTPUT FORMAT
========================
Return output strictly in the provided JSON schema (FinancialAnalysisOutput).
- Provide sources with page numbers for every critical figure.
- Explicitly list data gaps (missing metrics).
- Never fabricate numbers.
- Do not provide a final credit approval beyond APPROVE/REVIEW/DECLINE.

Tone: professional, analytical, evidence-based.
"""
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

class RiskFlag(BaseModel):
    type: Literal["critical", "warning", "info"]
    category: str
    metric: str
    value: Optional[float]
    message: str
    threshold: Optional[float] = None

class KeyRatios(BaseModel):
    current_ratio: Optional[float]
    quick_ratio: Optional[float]
    cash_ratio: Optional[float]
    debt_to_equity: Optional[float]
    debt_ratio: Optional[float]
    equity_ratio: Optional[float]
    roe: Optional[float]
    roa: Optional[float]
    profit_margin: Optional[float]
    gross_margin: Optional[float]
    operating_margin: Optional[float]

class Scores(BaseModel):
    liquidity_score: Optional[float]
    leverage_score: Optional[float]
    profitability_score: Optional[float]
    overall_financial_health: Optional[float]
    data_completeness_scoring_pct: float

class DataGap(BaseModel):
    metric: str
    reason: str

class FinancialAnalysisOutput(BaseModel):
    application_id: str
    company_name: Optional[str]
    fiscal_year: Optional[str]
    currency: Optional[str]
    extracted_figures: dict
    key_ratios: KeyRatios
    scores: Scores
    risk_flags: List[RiskFlag]
    data_gaps: List[DataGap]
    rationale: List[str]
    sources: List[str]
# AGENT LLM
print("1")
llm = ModelFactory.get("669a63646eb56306647e1091")  # GPT-4o Mini
llm.model_params.temperature = 0
llm.model_params.max_tokens = 4000
print("2")
loan_kb_obj = ModelFactory.get("6985a848037340c6889e1c51")
financial_calculator_obj = ModelFactory.get("69833ec2925f3f54f95756f2")

# CREATE AGENT
try:
  print("3")
  financial_agent = AgentFactory.create(
    name="Financial Analysis AgentV3",
    description="Calculates SME financial ratios and risk flags for loan applications",
    instructions=AGENT_INSTRUCTIONS,
    llm=llm,
    tools=[loan_kb_obj, financial_calculator_obj],
    expected_output=FinancialAnalysisOutput,

)
  print("4")
  # financial_agent.deploy()
  # financial_agent.save()
except Exception as e:
    print(f"✗ Creation failed: {str(e)}")
    print(f"\n🔍 Attempting to retrieve existing agent...")
FINANCIAL_AGENT_ID = financial_agent.id
print(f"\n Agent ID saved: {FINANCIAL_AGENT_ID}")

1
2
3


/usr/local/lib/python3.12/dist-packages/aixplain/factories/agent_factory/__init__.py:146: UserWarning: Deprecating 'llm_id', use `llm` to define the large language model in agents.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/aixplain/modules/model/connection.py:229: UserWarning: No action_scope is specified, by default all 1 actions will be included in Agent execution
  warnings.warn(


4

 Agent ID saved: 6985a902ff9be055415d4f98


In [ ]:
# إدخال الطلب بناءً على البيانات المستخرجة من الـ Log
test_input = """
Task: Conduct a full financial analysis for Application ID: APP-TEST-001.
Company: Stampede Auto Parts Inc.

Steps :

Search the Knowledge Base for all financial documents for this application.

Extract ALL available financial metrics mentioned in the 2019 data. This includes breakdown of current assets (like Cash, Inventory, A/R) and liabilities if found in the text or tables, to ensure maximum data completeness.

Pass all these figures to the Financial_Ratio_Caltool.

Provide the final analysis and your recommendation based on the full data set.
"""
fainancial_agent = AgentFactory.get("6985175bff9be055415d47be")
# تشغيل الـ Agent
try:
    print("Running Financial Analysis Agent...")
    response = financial_agent.run(test_input)
    print(response.data.intermediate_steps)
    if hasattr(response, 'data'):
        analysis_text = response.data
        print(analysis_text)
    else:
        print(response)
except Exception as e:
    print(f"An error occurred: {e}")

Running Financial Analysis Agent...
🤖  Financial Analysis AgentV3 | ⏳ Initialization
⚙️  Financial Analysis AgentV3 | GPT-4o Mini | ⏳
⚙️  Financial Analysis AgentV3 | Loan Applications KBV2 | ⏳
⚙️  Financial Analysis AgentV3 | GPT-4o Mini | ⏳
⚙️  Financial Analysis AgentV3 | cal_financial_ratios | ⏳
⚙️  Financial Analysis AgentV3 | GPT-4o Mini | ⏳
🤖  Financial Analysis AgentV3 | ⏳ Formatting Output
✅ Done | (30.2 s total | 6 API calls | $0.004297499999999999)
[{'agent': 'Financial Analysis AgentV3', 'input': '\nTask: Conduct a full financial analysis for Application ID: APP-TEST-001.\nCompany: Stampede Auto Parts Inc.\n\nSteps :\n\nSearch the Knowledge Base for all financial documents for this application.\n\nExtract ALL available financial metrics mentioned in the 2019 data. This includes breakdown of current assets (like Cash, Inventory, A/R) and liabilities if found in the text or tables, to ensure maximum data completeness.\n\nPass all these figures to the Financial_Ratio_Caltool.\

#Industry Agent

##NAICS CLASSIFIER TOOL

In [ ]:
# SECTION 1: CONFIGURATION & IDs

llm = ModelFactory.get("669a63646eb56306647e1091")
TAVILY_SEARCH_TOOL_ID = "6736411cf127849667606689"
LOAN_KB_ID = "6985a848037340c6889e1c51"

print(f"  LLM Model ID: {llm}")
print(f"  Search Tool ID: {TAVILY_SEARCH_TOOL_ID}")
print(f"  Knowledge Base ID: {LOAN_KB_ID}")

  LLM Model ID: Model: GPT-4o Mini by OpenAI (id=669a63646eb56306647e1091)
  Search Tool ID: 6736411cf127849667606689
  Knowledge Base ID: 6985a848037340c6889e1c51


In [ ]:

#  NAICS CLASSIFIER TOOL (DETERMINISTIC - NOT LLM)

def classify_naics(industry_description: str) -> dict:
    """
    Deterministic NAICS classification tool.
    Rule-based lookup - NOT LLM reasoning.

    Args:
        industry_description: Business description or industry name

    Returns:
        dict: NAICS classification with code, title, and sector
    """

    # Deterministic mapping database
    NAICS_DATABASE = {
        "automotive parts": {
            "code": "441310",
            "title": "Automotive Parts and Accessories Stores",
            "sector": "44-45",
            "sector_name": "Retail Trade",
            "subsector": "441",
            "subsector_name": "Motor Vehicle and Parts Dealers"
        },
        "auto parts": {
            "code": "441310",
            "title": "Automotive Parts and Accessories Stores",
            "sector": "44-45",
            "sector_name": "Retail Trade",
            "subsector": "441",
            "subsector_name": "Motor Vehicle and Parts Dealers"
        },
        "automobile wholesaler": {
            "code": "423110",
            "title": "Automobile and Other Motor Vehicle Merchant Wholesalers",
            "sector": "42",
            "sector_name": "Wholesale Trade",
            "subsector": "423",
            "subsector_name": "Merchant Wholesalers, Durable Goods"
        },
        "logistics": {
            "code": "484110",
            "title": "General Freight Trucking, Local",
            "sector": "48-49",
            "sector_name": "Transportation and Warehousing",
            "subsector": "484",
            "subsector_name": "Truck Transportation"
        },
        "freight": {
            "code": "484121",
            "title": "General Freight Trucking, Long-Distance",
            "sector": "48-49",
            "sector_name": "Transportation and Warehousing",
            "subsector": "484",
            "subsector_name": "Truck Transportation"
        },
        "trucking": {
            "code": "484110",
            "title": "General Freight Trucking, Local",
            "sector": "48-49",
            "sector_name": "Transportation and Warehousing",
            "subsector": "484",
            "subsector_name": "Truck Transportation"
        },
        "construction": {
            "code": "236220",
            "title": "Commercial and Institutional Building Construction",
            "sector": "23",
            "sector_name": "Construction",
            "subsector": "236",
            "subsector_name": "Construction of Buildings"
        },
        "software": {
            "code": "541512",
            "title": "Computer Systems Design Services",
            "sector": "54",
            "sector_name": "Professional, Scientific, and Technical Services",
            "subsector": "541",
            "subsector_name": "Professional, Scientific, and Technical Services"
        },
        "technology": {
            "code": "541512",
            "title": "Computer Systems Design Services",
            "sector": "54",
            "sector_name": "Professional, Scientific, and Technical Services",
            "subsector": "541",
            "subsector_name": "Professional, Scientific, and Technical Services"
        },
        "retail": {
            "code": "452000",
            "title": "General Merchandise Stores",
            "sector": "44-45",
            "sector_name": "Retail Trade",
            "subsector": "452",
            "subsector_name": "General Merchandise Stores"
        },
        "healthcare": {
            "code": "621111",
            "title": "Offices of Physicians",
            "sector": "62",
            "sector_name": "Health Care and Social Assistance",
            "subsector": "621",
            "subsector_name": "Ambulatory Health Care Services"
        },
        "restaurant": {
            "code": "722511",
            "title": "Full-Service Restaurants",
            "sector": "72",
            "sector_name": "Accommodation and Food Services",
            "subsector": "722",
            "subsector_name": "Food Services and Drinking Places"
        },
        "food service": {
            "code": "722511",
            "title": "Full-Service Restaurants",
            "sector": "72",
            "sector_name": "Accommodation and Food Services",
            "subsector": "722",
            "subsector_name": "Food Services and Drinking Places"
        },
        "plumbing": {
            "code": "238220",
            "title": "Plumbing, Heating, and Air-Conditioning Contractors",
            "sector": "23",
            "sector_name": "Construction",
            "subsector": "238",
            "subsector_name": "Specialty Trade Contractors"
        },
        "hvac": {
            "code": "238220",
            "title": "Plumbing, Heating, and Air-Conditioning Contractors",
            "sector": "23",
            "sector_name": "Construction",
            "subsector": "238",
            "subsector_name": "Specialty Trade Contractors"
        },
        "manufacturing": {
            "code": "332710",
            "title": "Machine Shops",
            "sector": "31-33",
            "sector_name": "Manufacturing",
            "subsector": "332",
            "subsector_name": "Fabricated Metal Product Manufacturing"
        },
        "warehousing": {
            "code": "493110",
            "title": "General Warehousing and Storage",
            "sector": "48-49",
            "sector_name": "Transportation and Warehousing",
            "subsector": "493",
            "subsector_name": "Warehousing and Storage"
        }
    }

    # Normalize input
    description_lower = industry_description.lower().strip()

    # Try exact match first
    for key, data in NAICS_DATABASE.items():
        if key in description_lower or description_lower in key:
            return {
                "status": "MATCHED",
                "input_description": industry_description,
                "naics_code": data["code"],
                "naics_title": data["title"],
                "sector_code": data["sector"],
                "sector_name": data["sector_name"],
                "subsector_code": data.get("subsector", ""),
                "subsector_name": data.get("subsector_name", ""),
                "classification_method": "DETERMINISTIC_LOOKUP"
            }

    # No match found
    return {
        "status": "NOT_MATCHED",
        "input_description": industry_description,
        "naics_code": "999990",
        "naics_title": "Unclassified Establishment",
        "sector_code": "99",
        "sector_name": "Unclassified",
        "subsector_code": "",
        "subsector_name": "",
        "classification_method": "DEFAULT_FALLBACK",
        "note": "Manual NAICS classification required"
    }

# Register NAICS Tool
try:
    naics_tool = ModelFactory.create_script_connection_tool(
        name="NAICS_Classifier2",
        description="Deterministic NAICS industry classification. Rule-based lookup, not LLM reasoning. Returns standardized NAICS code, title, and sector information.",
        code=classify_naics,
        # parameters={
        #     "industry_description": {
        #         "type": "string",
        #         "description": "Business description or industry name to classify",
        #         "required": True
        #     }
        # }
    )
    print(f"  Tool created: NAICS_Classifier")
    print(f"  Tool ID: {naics_tool.id}")
    NAICS_TOOL_ID = naics_tool.id

except Exception as e:
    print(f"  Tool creation failed: {str(e)}")
    print("  Attempting to use existing tool...")
    NAICS_TOOL_ID = None
    naics_tool = None



  Tool creation failed: Failed to create script connection tool: Failed to connect to 688779d8bfb8e46c273982ca - err.supplier_error - Name already exists
  Attempting to use existing tool...


##agent

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

"""
INDUSTRY RISK SIGNAL AGENT
ARD-compliant
Signal-based (Flags only) – No reports, no decisions
"""

# ========================
# OUTPUT SCHEMA
# ========================

class RiskFlag(BaseModel):
    type: Literal["critical", "warning", "info"]
    category: Literal[
        "political",
        "economic",
        "social",
        "technological",
        "environmental",
        "legal",
        "data_quality"
    ]
    metric: str
    message: str
    source: str

class DataGap(BaseModel):
    area: str
    reason: str

class IndustryRiskSignalOutput(BaseModel):
    application_id: str
    industry_name: Optional[str]
    naics_code: Optional[str]
    naics_title: Optional[str]
    geography: Optional[str]
    risk_flags: List[RiskFlag]
    data_gaps: List[DataGap]
    rationale: List[str]
    sources: List[str]

# ========================
# LOAD LLM
# ========================

llm = ModelFactory.get("669a63646eb56306647e1091")
llm.model_params.temperature = 0.2
llm.model_params.max_tokens = 6000

# ========================
# LOAD TOOLS
# ========================

search_tool = ModelFactory.get(TAVILY_SEARCH_TOOL_ID)
loan_kb = IndexFactory.get(LOAN_KB_ID)

naics_tool = None
if NAICS_TOOL_ID:
    try:
        naics_tool = ModelFactory.get(NAICS_TOOL_ID)
    except Exception:
        naics_tool = None

tools = [loan_kb, search_tool]
if naics_tool:
    tools.append(naics_tool)

# ========================
# AGENT INSTRUCTIONS
# ========================

INDUSTRY_AGENT_INSTRUCTIONS = """
You are the Industry Risk Signal Extraction & Market Intelligence Agent for BR Finance.

Your goal is to detect, classify, and emit industry-related risk signals using a strict
KB-First, Web-Second methodology while producing a structured PESTEL risk report.

==================================================
PRIMARY MISSION
==================================================
When given an Application ID (e.g., APP-TEST-001), you must:
• Discover the company industry from the Knowledge Base FIRST.
• Extract NAICS + geography from KB when available.
• Perform a PESTEL-based industry risk assessment.
• Use the web ONLY if the KB lacks sufficient industry/market risk data.

==================================================
CORE OPERATIONAL FLOW (CRITICAL)
==================================================

STEP 1 — INTERNAL DISCOVERY (MANDATORY FIRST)
You MUST query the Loan Applications Knowledge Base BEFORE doing anything else.

KB Query formats:
1) Primary discovery query:
   {"data": "<application_id>", "dataType": "application_id"}

2) If industry details are incomplete, run an additional KB semantic search:
   {"data": "<application_id> industry business NAICS", "dataType": "text"}

From KB results, EXTRACT:
• Company name
• Industry name / description
• NAICS code (if present)
• Geography / operating location (REAL business location)
• ANY existing industry, market, or PESTEL-related commentary

IMPORTANT:
If the KB returns usable industry information, you MUST treat it as the primary source.

--------------------------------------------------

STEP 2 — NAICS COMPLETION (ONLY IF MISSING)
If NAICS is not present in the KB:
Use NAICS classification tool:
Input format:
{"industry_description": "..."}

Do NOT guess NAICS without this tool.

--------------------------------------------------

STEP 3 — ADAPTIVE WEB SEARCH (KB-SECOND)
You may use web search ONLY if the KB does not provide sufficient
industry / market / PESTEL risk signals.

Rules:
• NEVER assume geography.
• ALWAYS use the geography extracted from KB.
• Search for LOCATION-SPECIFIC industry risks.

Example Queries (Logic only, see Tool Formats for syntax):
"Automotive aftermarket industry risks in Michigan USA"
"PESTEL analysis for food distribution industry Netherlands"

--------------------------------------------------

STEP 4 — SIGNAL SYNTHESIS
Combine:
• KB insights (primary signals)
• Web insights (supporting signals)

If KB already mentions a PESTEL factor:
→ Treat it as HIGH CONFIDENCE and support it with web evidence if needed.

==================================================
RISK SIGNAL SCOPE (PESTEL)
==================================================
Extract and classify risks under:

• Political
• Economic
• Social
• Technological
• Environmental
• Legal

==================================================
CRITICAL TOOL FORMATS (STRICT)
==================================================

1) Loan Applications KB:
   Input: {"data": "ID_HERE", "dataType": "application_id"}

2) KB semantic search (Backup):
   Input: {"data": "ID_HERE industry business NAICS", "dataType": "text"}

3) NAICS tool:
   Input: {"industry_description": "..."}

4) Web Search (Tavily):
   Input Format: {"query": "<search string>"}

   CRITICAL RULES FOR WEB SEARCH:
   - Input MUST be a JSON object (Dictionary).
   - The object MUST have EXACTLY one key named "query".
   - The value MUST be a simple string.
   - DO NOT add extra keys like "search_depth", "topic", or "days".

   CORRECT Example:
   {"query": "Automotive aftermarket risks in Michigan 2024"}

   INCORRECT Examples (DO NOT USE):
   "Automotive risks..." (This is a string, not a JSON object)
   {"text": "..."} (Wrong key name)
   {"query": "...", "search_depth": "basic"} (Too many keys)

==================================================
OUTPUT FORMATTING — STRICT COMPLIANCE
==================================================
Return ONLY valid JSON matching `IndustryRiskSignalOutput`.

Fill ALL fields:

1) application_id → must match input
2) industry_name → from KB
3) naics_code → confirmed or tool-derived
4) geography → REAL operating location from KB
5) risk_flags → list of objects:
   {
     "type": "risk/opportunity",
     "category": "Political/Economic/Social/Technological/Environmental/Legal",
     "metric": "short label",
     "message": "clear explanation of the risk",
     "source": "KB: Document Name" OR "Web: URL"
   }
6) data_gaps → missing PESTEL factors
7) rationale → why these risks matter for THIS business
8) sources → list ALL unique KB documents and URLs used

DO NOT output markdown.
DO NOT simplify or omit fields.
"""

# ========================
# CREATE AGENT
# ========================

industry_risk_agent = AgentFactory.create(
    name="Industry PESTEL Analysis AgentV2",
    description="Extracts structured industry risk flags (PESTEL-based) from KB and conditional web evidence. No reports, no decisions.",
    instructions=INDUSTRY_AGENT_INSTRUCTIONS,
    llm=llm,
    tools=tools,
    expected_output=IndustryRiskSignalOutput
)
# industry_risk_agent.deploy()
INDUSTRY_AGENT_ID = industry_risk_agent.id
print(f"Industry Risk Signal Agent created. ID: {INDUSTRY_AGENT_ID}")


Industry Risk Signal Agent created. ID: 6985b147ff9be055415d526b


In [ ]:
import json
from IPython.display import display, Markdown

# ============================================================================
# 🧪 INDUSTRY AGENT TEST CELL
# ============================================================================

# 1. إعداد نص الطلب (Prompt)
# نستخدم نفس الصيغة التي نجحت في الـ Logs لضمان الاستمرارية
test_prompt = """
Perform a PESTEL Analysis for Application ID: APP-STAMPEDE-004.
Company: Stampede Auto Parts Inc.
Industry Description: Automotive parts distributor and manufacturer

Mandatory Steps:
1. Search KB for the application details (extract geography & context).
2. Classify NAICS using the dedicated tool.
3. Conduct web research for PESTEL factors one by one.
4. Synthesize everything into the final IndustryRiskSignalOutput JSON format.

Note: When using search tools, only provide the query string.
"""

print(f"🚀 Starting Optimized Industry Analysis...")

try:
    # 2. تشغيل الإيجنت
    # max_iterations=20 ضروري لأن الإيجنت يقوم ببحث متعدد (Political, Economic, etc.)
    response = industry_risk_agent.run(test_prompt, max_iterations=20)

    print("-" * 60)
    print("✅ ANALYSIS COMPLETE")
    print("-" * 60)

    # 3. استخراج البيانات وتنسيقها (JSON Pretty Print)
    # نحاول الوصول إلى المخرج النهائي بغض النظر عن نوع الكائن العائد
    if hasattr(response, 'data') and hasattr(response.data, 'output'):
        final_output = response.data.output
    elif hasattr(response, 'output'):
        final_output = response.output
    else:
        final_output = response

    # 4. عرض النتيجة بشكل مقروء
    if isinstance(final_output, (dict, list)):
        # إذا كان المخرج قاموساً (وهو المتوقع)، نطبعه بتنسيق JSON
        print(json.dumps(final_output, indent=2, ensure_ascii=False))
    elif hasattr(final_output, 'model_dump'):
        # إذا كان Pydantic Model
        print(json.dumps(final_output.model_dump(), indent=2, ensure_ascii=False))
    else:
        # محاولة تحويل النص إلى JSON إذا كان string
        try:
            parsed = json.loads(str(final_output))
            print(json.dumps(parsed, indent=2, ensure_ascii=False))
        except:
            # إذا فشل كل شيء، اطبع النص كما هو
            print(str(final_output))

except Exception as e:
    print(f"❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()

🚀 Starting Optimized Industry Analysis...
🤖  Industry PESTEL Analysis AgentV2 | ⏳ Initialization
⚙️  Industry PESTEL Analysis AgentV2 | GPT-4o Mini | ⏳
⚙️  Industry PESTEL Analysis AgentV2 | Loan Applications KBV2 | ⏳
⚙️  Industry PESTEL Analysis AgentV2 | GPT-4o Mini | ⏳
🤖  Industry PESTEL Analysis AgentV2 | ⏳ Formatting Output
✅ Done | (32.2 s total | 4 API calls | $0.0035263499999999993)
------------------------------------------------------------
✅ ANALYSIS COMPLETE
------------------------------------------------------------
{
  "application_id": "APP-STAMPEDE-004",
  "industry_name": "Automotive parts distributor and manufacturer",
  "naics_code": "441310",
  "geography": "Michigan and Indiana",
  "risk_flags": [
    {
      "type": "risk",
      "category": "Political",
      "metric": "Minimum wage changes",
      "message": "Changes in minimum wage laws in Indiana may affect profitability.",
      "source": "KB: APP-TEST-001_page003_PESTEL_ANALYSIS"
    },
    {
      "type": 

#Collateral Agent

##collateral_policy_evaluator

In [ ]:
from aixplain.factories import ModelFactory

# 1. تعريف الدالة (المنطق الحسابي)
def collateral_policy_evaluator(
    asset_type: str,
    doc_value: float,
    location_risk: str,
    inspection_age_months: int,
    condition: str,
    lien_position: str
) -> dict:
    """
    Rule-based haircut calculator (per ARD collateral policy).
    """
    # تحسين بسيط: تحويل المدخلات لنوعها الصحيح لضمان عدم الفشل
    try:
        doc_value = float(doc_value)
        inspection_age_months = int(inspection_age_months)
    except:
        return {"error": "Invalid numeric input"}

    # تنظيف النصوص
    asset_type = str(asset_type).upper()
    if "INVENTORY" in asset_type: asset_type = "INVENTORY"
    elif "RECEIVABLE" in asset_type: asset_type = "RECEIVABLES"
    elif "ESTATE" in asset_type: asset_type = "REAL_ESTATE"
    elif "EQUIPMENT" in asset_type: asset_type = "EQUIPMENT"
    else: asset_type = "OTHER"

    location_risk = str(location_risk).upper() if location_risk else "MEDIUM"
    condition = str(condition).upper() if condition else "AVERAGE"
    lien_position = str(lien_position).upper() if lien_position else "FIRST"

    # القواعد
    rules = {
        "REAL_ESTATE": {"base_advance": 0.65, "max_inspection_age": 12},
        "EQUIPMENT":   {"base_advance": 0.55, "max_inspection_age": 9},
        "INVENTORY":   {"base_advance": 0.50, "max_inspection_age": 6},
        "RECEIVABLES": {"base_advance": 0.70, "max_inspection_age": 3},
        "CASH":        {"base_advance": 0.90, "max_inspection_age": 18},
        "OTHER":       {"base_advance": 0.40, "max_inspection_age": 6},
    }

    rule = rules.get(asset_type, rules["OTHER"])
    advance = rule["base_advance"]

    # تطبيق الخصومات
    if "HIGH" in location_risk: advance -= 0.1
    elif "LOW" in location_risk: advance += 0.05

    if inspection_age_months > rule["max_inspection_age"]: advance -= 0.1

    if "STRONG" in condition or "GOOD" in condition: advance += 0.05
    elif "WEAK" in condition or "POOR" in condition: advance -= 0.1

    if "FIRST" not in lien_position: advance -= 0.15

    # الحدود (Cap/Floor)
    advance = max(0.1, min(advance, 0.9))
    eligible_value = doc_value * advance
    haircut = 1 - advance

    return {
        "asset_type": asset_type,
        "doc_value": doc_value,
        "advance_rate": round(advance, 2),
        "haircut": round(haircut, 2),
        "eligible_value": round(eligible_value, 2)
    }

# 2. النشر (بدون Parameters يدوية - هذا هو الحل السحري)
try:
    print("⏳ Re-deploying Collateral Tool (Auto-Schema)...")

    collateral_tool = ModelFactory.create_script_connection_tool(
        name="Collateral_Policy_Calc",
        description="Calculates eligible collateral value. Inputs: asset_type, doc_value, location_risk, inspection_age_months, condition, lien_position.",
        code=collateral_policy_evaluator
        # ❌ حذفنا parameters={} تماماً لنترك المكتبة تعالج الأنواع بمرونة
    )

    # collateral_tool.deploy()

    print(f"✅ Tool Fixed & Deployed!")
    print(f"🆔 New Tool ID: {collateral_tool.id}")

    # احفظ هذا المتغير للخطوة التالية
    NEW_POLICY_TOOL_ID = collateral_tool.id

except Exception as e:
    print(f"❌ Deployment Failed: {str(e)}")

⏳ Re-deploying Collateral Tool (Auto-Schema)...
❌ Deployment Failed: Failed to create script connection tool: Failed to connect to 688779d8bfb8e46c273982ca - err.supplier_error - Name already exists


##agent

In [ ]:
from aixplain.factories import ModelFactory, AgentFactory, IndexFactory
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

# ============================================================================
# 1. SETUP & CONFIGURATION
# ============================================================================

LOAN_KB_ID = "6985a848037340c6889e1c51"
# تأكد أن COLLATERAL_POLICY_TOOL_ID معرف سابقاً لديك
# COLLATERAL_POLICY_TOOL_ID = "..."

# تحميل الـ LLM
try:
    # استخدام موديل قوي للتحليل (Claude 3.5 Sonnet أو GPT-4o)
    llm = ModelFactory.get("669a63646eb56306647e1091")
    llm.model_params.temperature = 0.0  # Zero temperature for deterministic output
    print("✓ LLM loaded")
except Exception as e:
    print(f"✗ LLM load error: {e}")
    raise

# تحميل الـ Tools
try:
    loan_kb = IndexFactory.get(LOAN_KB_ID)
    policy_tool = ModelFactory.get("6983b6907f68903010ac901d")
    tools_for_agent = [loan_kb, policy_tool]
    print(f"✓ Tools loaded: KB ({loan_kb.name}) + Policy Tool")
except Exception as e:
    print(f"✗ Tool load error: {e}")
    raise


# ============================================================================
# 2. DEFINE OUTPUT SCHEMA (PYDANTIC)
# ============================================================================

class CollateralItem(BaseModel):
    asset_type: str = Field(..., description="Type of asset (Inventory, Receivables, Real Estate, etc.)")
    description: str = Field(..., description="Brief description of the asset")
    book_value: float = Field(..., description="The gross value found in the documents")
    valuation_date: Optional[str] = Field(None, description="Date of valuation if available")
    lien_position: str = Field("Unknown", description="First, Second, or Subordinated")

    # Policy Tool Results
    advance_rate: Optional[float] = Field(None, description="The haircut rate applied (0.0 to 1.0)")
    eligible_value: float = Field(..., description="Value after applying haircut")

    source: Literal["kb", "user_provided"] = Field("kb", description="Where this data came from")
    notes: Optional[str] = Field(None, description="Any warnings or assumptions made")

class CoverageAnalysis(BaseModel):
    total_exposure: float = Field(..., description="Total requested loan amount")
    total_eligible_collateral: float = Field(..., description="Sum of all eligible collateral values")
    coverage_ratio: float = Field(..., description="Total Eligible / Total Exposure")
    surplus_deficit: float = Field(..., description="Eligible Value - Total Exposure")
    status: Literal["Fully Secured", "Partially Secured", "Unsecured"]

class Recommendation(BaseModel):
    decision: Literal["Approve", "Conditional Approve", "Reject"]
    conditions: List[str] = Field(default_factory=list, description="Conditions if approved")
    risk_factors: List[str] = Field(default_factory=list, description="identified risks")

class CollateralReport(BaseModel):
    application_id: str
    borrower_name: str
    collateral_items: List[CollateralItem]
    coverage: CoverageAnalysis
    recommendation: Recommendation

print("✓ Output Schema Defined (CollateralReport)")


# ============================================================================
# 3. AGENT INSTRUCTIONS (OPTIMIZED)
# ============================================================================


# ============================================================================
# 3. AGENT INSTRUCTIONS (FINAL OPTIMIZED VERSION)
# ============================================================================

COLLATERAL_AGENT_INSTRUCTIONS = """
You are the Collateral Analyst Agent.

Your role is to perform collateral analysis for loan applications.
You MUST strictly follow the rules below.

────────────────────────────────────────
GENERAL RULES (MANDATORY)
────────────────────────────────────────

1. You MUST retrieve collateral values ONLY from the Loan Applications KB.
   - Never assume values.
   - Never use user-provided numbers unless they are explicitly found in the KB search result text.

2. **CRITICAL: TOOL INPUT FORMAT**
   The 'Loan Applications KB' tool REQUIRES a specific JSON object structure.
   You MUST call it with this EXACT format:

   {
     "data": "<YOUR SEARCH QUERY HERE>",
     "dataType": "text"
   }

   - `data`: The search string (e.g., "APP-TEST-001 inventory").
   - `dataType`: MUST be the string "text". DO NOT use "query" or anything else.

3. NEVER send null or None to any tool input.

4. If a KB search fails (returns empty or error):
   - Retry at least ONCE with a shorter query string inside the "data" field.
   - Example retry: {"data": "APP-TEST-001 collateral", "dataType": "text"}

────────────────────────────────────────
KB QUERY STRATEGY
────────────────────────────────────────

Construct your queries to find asset values.
Examples of valid tool calls:

- To find inventory:
  `{"data": "APP-TEST-001 inventory value amount", "dataType": "text"}`

- To find receivables:
  `{"data": "APP-TEST-001 accounts receivable aging value", "dataType": "text"}`

- To find general collateral info:
  `{"data": "APP-TEST-001 collateral security table", "dataType": "text"}`

────────────────────────────────────────
DATA EXTRACTION RULES (SMART & ADAPTIVE)
────────────────────────────────────────

When extracting 'Book Value' for collateral assets (Inventory, Receivables, etc.), apply this logic strictly to avoid undervaluation:

1. **SOURCE PRIORITY:**
   - **PRIORITIZE** values found in 'Financial Statements', 'Balance Sheet', or 'Financial Tables' sections (often indicated by metadata `content_type="FINANCIAL_TABLE"`).
   - **BE CAUTIOUS** of values found in 'Security', 'Collateral', 'Facility Letter', or 'Covenants' sections.

2. **DISTINGUISH "VALUE" VS "CAP":**
   - **Book Value (Target):** Usually a specific number representing the asset's actual worth on the books (e.g., "Inventory: $4,721,000").
   - **Lending Cap (Avoid):** Often accompanied by terms like "Max", "Up to", "Limited to", or "Capped at" (e.g., "Inventory up to $2,500,000").
   - **DO NOT** use the Lending Cap as the Book Value.

3. **THE "HIGHER VALUE" HEURISTIC:**
   - If you find two conflicting numbers for the same asset (e.g., $4.7M and $2.5M), the **HIGHER** number is almost always the Book Value, and the **LOWER** number is the Bank's Exposure Limit.
   - Select the **HIGHER** number as the input for 'book_value' before applying the policy tool.

4. **ASSET MAPPING & FORMATTING:**
   - Standardize `asset_type` to: "Inventory", "Receivables", "Real Estate", "Equipment", or "Cash".
   - Ensure `source` is always set to "kb".
   - Include a brief `description` extracted from the document context.

────────────────────────────────────────
POLICY APPLICATION
────────────────────────────────────────

If numeric collateral values are found:
- You MUST apply `CollateralPolicyTool`.
- Pass the **Book Value** (the higher number identified above) to the tool.
- The tool will calculate the `advance_rate` and `eligible_value` (applying haircuts).

If policy tool is unavailable or fails:
- Set advance_rate = null
- Set eligible_value = book_value
- Add a note explaining the assumption.

────────────────────────────────────────
COVERAGE ANALYSIS
────────────────────────────────────────

Compute:
- `total_eligible_collateral` (Sum of eligible values from policy tool)
- `coverage_ratio` = total_eligible_collateral / total_exposure
- `surplus_deficit` = total_eligible_collateral - total_exposure

Coverage status rules:
- coverage_ratio >= 1.0 → "Fully Secured"
- 0 < coverage_ratio < 1.0 → "Partially Secured"
- coverage

"""

# ============================================================================
# 4. CREATE AGENT
# ============================================================================

print("\n🤖 Creating Collateral Agent...")

try:
    collateral_agent = AgentFactory.create(
        name="Collateral Analyst AgentV2 ",
        description="Evaluates collateral coverage using KB data and policy tools, returning structured JSON.",
        instructions=COLLATERAL_AGENT_INSTRUCTIONS,
        llm=llm,
        tools=tools_for_agent,
        expected_output=CollateralReport
    )
    # collateral_agent.deploy()

    print(f"  ✓ Agent Created Successfully!")
    # print(f"    ID: {collateral_agent.id}")
    print(f"    Name: {collateral_agent.name}")

    COLLATERAL_AGENT_ID = collateral_agent.id
    print(f"\n👉 SAVE THIS ID: COLLATERAL_AGENT_ID = \"{COLLATERAL_AGENT_ID}\"")

except Exception as e:
    print(f"✗ Creation failed: {e}")


✓ LLM loaded
✓ Tools loaded: KB (Loan Applications KBV2) + Policy Tool
✓ Output Schema Defined (CollateralReport)

🤖 Creating Collateral Agent...
  ✓ Agent Created Successfully!
    Name: Collateral Analyst AgentV2 

👉 SAVE THIS ID: COLLATERAL_AGENT_ID = "6985b275ae1159fcbe5bbebb"


In [ ]:

# SECTION 6: TEST CELL
print("\n" + "="*70)
print(" SECTION 6: TESTING THE AGENT")
print("="*70)

# test_prompt = """
# Analyze collateral coverage for application APP-TEST-001.

# 1. Query the Loan Applications Knowledge Base for documents with
#    metadata.application_id = "APP-TEST-001".

# 2. From the KB, extract:
#    - Borrower name.
#    - All credit facilities (Term Loan and Line of Credit) and their amounts.
#    - All pledged collateral assets, including:
#      • Inventory
#      • Accounts Receivable (AR)
#      • Real estate at 123 Alphabet Lane

# 3. For each collateral asset:
#    - Extract the documented value (doc_value) from the KB.
#    - Determine asset_type based on description.
#    - If qualitative parameters are not explicitly stated in the KB,
#      apply conservative defaults as permitted by your instructions:
#        • location_risk = MEDIUM
#        • condition = AVERAGE
#        • lien_position = FIRST
#    - Calculate inspection_age_months from any site visit or appraisal dates if available.

# 4. Call CollateralPolicyTool for each asset where doc_value is available.
#    Pass all parameters as individual arguments.
#    Do NOT perform any manual calculations.

# 5. Aggregate eligible collateral values, assess coverage against total exposure,
#    and flag any documentation gaps or stale valuations.

# Return ONLY a valid JSON object in the exact format specified in your instructions.


# """
test_prompt = """
Perform a collateral analysis for application APP-TEST-001.

Use the following data ONLY if found in KB:
- Inventory value: 4721000
- Accounts Receivable value: 3351000
- Total exposure: 6500000

Apply CollateralPolicyTool where possible.

Return the result STRICTLY as valid JSON matching the CollateralReport schema.
"""
# SECTION 6: DEBUG TEST
print("🚀 Running Debug Analysis...")
try:
    # collateral_agent = AgentFactory.get("69848934ae1159fcbe5ba2d2")
    # أضفنا verbose=True وقمنا بتغيير طريقة الاستدعاء قليلاً
    response = collateral_agent.run(test_prompt)

    # فحص كائن الاستجابة بالكامل
    print(f"Status: {response.status}")
    if response.status == "FAILED":
        # محاولة طباعة تفاصيل الخطأ من داخل الكائن
        print(f"Error Detail: {response}")
    else:
        print(f"Output: {response.data}")
except Exception as e:
    print(f"General Exception: {str(e)}")

# ============================================================================
# SECTION 7: SAVE IDs
# ============================================================================
print("\n" + "="*70)
print(" SECTION 7: SAVE IDS")
print("="*70)
# print(f"COLLATERAL_AGENT_ID = \"{COLLATERAL_AGENT_ID}\"")
# print(f"COLLATERAL_POLICY_TOOL_ID = \"{COLLATERAL_POLICY_TOOL_ID}\"")
print("="*70)
print(" COLLATERAL AGENT SETUP COMPLETE")
print("="*70)


 SECTION 6: TESTING THE AGENT
🚀 Running Debug Analysis...
🤖  Collateral Analyst AgentV2  | ⏳ Initialization
⚙️  Collateral Analyst AgentV2  | GPT-4o Mini | ⏳
⚙️  Collateral Analyst AgentV2  | Loan Applications KBV2 | ⏳
⚙️  Collateral Analyst AgentV2  | GPT-4o Mini | ⏳
⚙️  Collateral Analyst AgentV2  | Loan Applications KBV2 | ✓ The output of the function 'search' to the input '{'data': 'APP-TEST-001 accounts receivable value amount', 'dataType': 'text'}' is: [{'score': 0.80141354, 'data': 'ADDITIONAL FINANCIAL ANALYSIS\n\n##...
⚙️  Collateral Analyst AgentV2  | GPT-4o Mini | ⏳
🤖  Collateral Analyst AgentV2  | ⏳ Formatting Output
✅ Done | (41.0 s total | 10 API calls | $0.00940635)
Status: SUCCESS
Output: AgentResponseData(input={'input': '\nPerform a collateral analysis for application APP-TEST-001.\n\nUse the following data ONLY if found in KB:\n- Inventory value: 4721000\n- Accounts Receivable value: 3351000\n- Total exposure: 6500000\n\nApply CollateralPolicyTool where possible.\n\

# Create SWOT analysis agent

In [ ]:
from aixplain.factories import AgentFactory
from pydantic import BaseModel, Field
from typing import List, Literal, Optional

# ==========================================
# 1. تعريف هيكل المخرجات (Schema) بدقة
# ==========================================

class ManagementAssessment(BaseModel):
    overall_quality: Literal["Strong", "Adequate", "Weak"] = Field(..., description="Overall assessment of management capability")
    key_strengths: List[str] = Field(..., description="List of specific management strengths")
    key_weaknesses: List[str] = Field(..., description="List of specific management weaknesses (e.g., succession, thin team)")
    rationale: str = Field(..., description="Detailed justification for the rating")

class SWOT(BaseModel):
    strengths: List[str]
    weaknesses: List[str]
    opportunities: List[str]
    threats: List[str]

class SupplyChainRisk(BaseModel):
    supplier_concentration: Literal["Low", "Moderate", "High"] = Field(..., description="Risk level based on supplier dependency")
    customer_concentration: Literal["Low", "Moderate", "High"] = Field(..., description="Risk level based on customer dependency (e.g. >20% revenue from one client)")
    mitigation_factors: Optional[str] = Field(None, description="Any factors that reduce these risks")
    risk_notes: str

class BusinessManagementOutput(BaseModel):
    company_name: str
    management_assessment: ManagementAssessment
    swot_analysis: SWOT
    supply_chain: SupplyChainRisk
    risk_flags: List[str] = Field(..., description="Specific red flags for credit consideration")
    qualitative_summary: str = Field(..., description="Executive summary for the credit memo")

# ==========================================
# 2. التعليمات المحسنة (Credit Logic Instructions)
# ==========================================

INSTRUCTIONS = """
You are a Senior Credit Analyst Agent for a bank.

GOAL: Search the Knowledge Base (KB) for qualitative data and analyze Management, SWOT, and Concentration Risks.

### STEP 1: DATA GATHERING (Use Loan Applications KB)
You MUST search the KB using this JSON format: {"data": "<application_id> <keywords>", "dataType": "text"}

STEP 1: SEARCH
- Search for Management: {"data": "[APP_ID] management owner experience", "dataType": "text"}
- Search for Business: {"data": "[APP_ID] business customers suppliers", "dataType": "text"}
- Search for Strategy: {"data": "[APP_ID] strategy competition", "dataType": "text"}

### STEP 2: ANALYSIS LOGIC
### ANALYSIS LOGIC (Apply these rules):

1. **Management Analysis:**
   - Assess experience, tenure, and succession planning.
   - **Rule:** If the company is family-owned with no clear successor mentioned or implied, flag "Succession Risk".
   - **Rule:** If the management team has < 5 years experience, rate as "Weak" or "Adequate" at best.

2. **Supply Chain & Concentration Risk:**
   - **Customer Concentration:** If a single customer accounts for >20% of revenue, mark Customer Concentration as "High".
   - **Supplier Concentration:** If the business relies on 1-2 main suppliers, mark Supplier Concentration as "High".

3. **SWOT Analysis:**
   - Extract internal Strengths/Weaknesses and external Opportunities/Threats.
   - Be specific (e.g., instead of "Good customers", say "Strong relationship with Ford dealerships").

4. **Risk Flags:**
   - Explicitly list any factors that could cause loan default (e.g., "High dependency on Ford", "Lack of succession plan").

### OUTPUT FORMAT:
- You must return a valid JSON object matching the `BusinessManagementOutput` schema.
- Do NOT output markdown text outside the JSON.
- Ensure all fields are filled based on the input text.
"""

# ==========================================
# 3. إنشاء العميل (Agent Creation)
# ==========================================
loan_kb = IndexFactory.get(LOAN_KB_ID)
print("🚀 Creating Business Analysis Agent...")
try:
    business_management_agent = AgentFactory.create(
        name="Business and Management AnalystV2",
        description="Analyzes management quality, SWOT, and concentration risks for credit memos.",
        instructions=INSTRUCTIONS,
        tools=[loan_kb],
        llm=ModelFactory.get("669a63646eb56306647e1091"), # GPT-4o Mini
        expected_output=BusinessManagementOutput
    )
    business_management_agent.deploy()
    print(f"✅ Agent Created: {business_management_agent.id}")
except Exception as e:
    print(f"❌ Error creating agent: {e}")



🚀 Creating Business Analysis Agent...
✅ Agent Created: 6985b388ae1159fcbe5bbf43


In [ ]:
import json
from IPython.display import display, Markdown

# ============================================================================
# 🧪 BUSINESS AGENT TEST CELL (Autonomous Mode)
# ============================================================================

# 1. تحديد معرف الطلب (يجب أن يطابق ما تم تخزينه في الـ KB في خطوة الـ OCR)
# في اللوج السابق، قمت بتخزين الملف باسم: APP-TEST-001
TARGET_APP_ID = "APP-TEST-001"

# 2. صياغة الأمر (Prompt)
# لاحظ أننا لا نعطيه النص، بل نأمره بالبحث عنه
test_prompt = f"""
Perform a comprehensive Business & Management Analysis for Application ID: {TARGET_APP_ID}.

Task:
1. Search the Knowledge Base for management bios, ownership structure, and business overview for this application.
2. Identify key risks related to succession, experience, and concentration (customers/suppliers).
3. Construct a SWOT analysis based strictly on the retrieved documents.

Output:
Return a strictly valid JSON object matching the BusinessManagementOutput schema.
"""

print(f"🚀 Starting Autonomous Business Analysis for {TARGET_APP_ID}...")

try:
    # 3. تشغيل العميل
    # لا نحتاج max_iterations عالية جداً هنا، 5-10 كافية لأنه بحث نصي مباشر
    response = business_management_agent.run(test_prompt, max_iterations=10)

    print("-" * 60)
    print("✅ ANALYSIS COMPLETE")
    print("-" * 60)

    # 4. استخراج وتنسيق النتيجة
    final_output = None

    # محاولة استخراج البيانات من الهياكل المحتملة المختلفة
    if hasattr(response, 'data') and hasattr(response.data, 'output'):
        final_output = response.data.output
    elif hasattr(response, 'output'):
        final_output = response.output
    else:
        final_output = response

    # 5. عرض النتيجة بتنسيق JSON مقروء
    if isinstance(final_output, (dict, list)):
        print(json.dumps(final_output, indent=2, ensure_ascii=False))
    elif hasattr(final_output, 'model_dump'):
        print(json.dumps(final_output.model_dump(), indent=2, ensure_ascii=False))
    else:
        # محاولة تحويل النص إلى JSON إذا كان string
        try:
            parsed = json.loads(str(final_output))
            print(json.dumps(parsed, indent=2, ensure_ascii=False))
        except:
            # طباعة النص كما هو إذا فشل التحويل
            print("⚠️ Raw Output (Not JSON):")
            print(str(final_output))

except Exception as e:
    print(f"❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()

🚀 Starting Autonomous Business Analysis for APP-TEST-001...
🤖  Business and Management Analyst | ⏳ Initialization
⚙️  Business and Management Analyst | GPT-4o Mini | ⏳
⚙️  Business and Management Analyst | Loan Applications KB | ⏳
⚙️  Business and Management Analyst | GPT-4o Mini | ⏳
⚙️  Business and Management Analyst | Loan Applications KB | ⏳
⚙️  Business and Management Analyst | GPT-4o Mini | ⏳
🤖  Business and Management Analyst | ⏳ Formatting Output
🤖  Business and Management Analyst | ✓ Formatting Output
✅ Done | (32.6 s total | 6 API calls | $0.0053435999999999996)
------------------------------------------------------------
✅ ANALYSIS COMPLETE
------------------------------------------------------------
{
  "management_analysis": {
    "management_team": [
      {
        "name": "Rolan Ramsay",
        "position": "CEO",
        "experience_years": 40
      },
      {
        "name": "Robyn Ramsay",
        "position": "CFO/COO",
        "experience_years": 10
      },
      {

# Create GCC and ESG Agent

In [ ]:
from aixplain.factories import AgentFactory, ModelFactory, IndexFactory
from pydantic import BaseModel, Field
from typing import List, Literal, Optional

# ==========================================
# 1. SETUP TOOLS
# ==========================================
LOAN_KB_ID = "6985a848037340c6889e1c51"
try:
    loan_kb = IndexFactory.get(LOAN_KB_ID)
    print(f"✅ Tools loaded: {loan_kb.name}")
except:
    print("❌ Error loading KB")

# ==========================================
# 2. UPDATED SCHEMA (FLEXIBLE)
# ==========================================
class GCCComplianceOutput(BaseModel):
    # Context
    operating_region: str = Field(..., description="Detected region: 'GCC' or 'International/USA'")

    # 1. Activity Check
    is_compliant_activity: bool = Field(..., description="True if business activity is allowed (No alcohol, gambling, etc)")

    # 2. Regulatory Status (Adaptive)
    license_status: Literal["Valid", "Expired", "Not Found"] = Field(..., description="Status of CR, Trade License, or Incorporation Cert")
    tax_status: Literal["Compliant", "Non-Compliant", "Unknown"] = Field(..., description="VAT/Zakat for GCC, or Tax Returns for Intl")

    # 3. Labor & Social (Adaptive)
    labor_compliance: Literal["Compliant", "Non-Compliant", "Unknown"] = Field(..., description="Nitaqat/WPS for GCC, or Labor Laws for Intl")

    # 4. AML & Sanctions
    aml_sanction_check: Literal["Clear", "Flagged", "Manual Review Required"]

    # 5. Summary
    overall_compliance_rating: Literal["Pass", "Conditional Pass", "Fail"]
    recommendation_notes: str
    missing_documents: List[str]

# ==========================================
# 3. INTELLIGENT INSTRUCTIONS
# ==========================================

COMPLIANCE_INSTRUCTIONS = """
You are a Senior Compliance Auditor for BR Finance.
Your goal is to audit loan applications against regulatory standards by adaptively analyzing borrower documents.

==================================================
CRITICAL: SEARCH & RETRIEVAL STRATEGY
==================================================
You MUST extract the ACTUAL Application ID from the prompt.
To ensure you see the WHOLE file (and not just fragments), use this EXACT tool input:
{"data": "<actual_id>", "dataType": "application_id"}

Rule: Execute this search once. If the initial results are broad, perform a second semantic search to target missing sections:
{"data": "<actual_id> covenants taxes compliance labor", "dataType": "text"}

==================================================
STEP 1: DYNAMIC GEOGRAPHY DETECTION
==================================================
Do not assume the region. Scan the retrieved text for addresses or country names:
- Keywords: "USA", "MI", "Detroit", "London", "Europe" -> Set operating_region to 'International/USA'.
- Keywords: "KSA", "Saudi", "UAE", "Dubai", "Riyadh", "GCC" -> Set operating_region to 'GCC'.

==================================================
STEP 2: SMART CROSS-DOCUMENT EXTRACTION
==================================================
Analyze all returned segments using "Equivalent Standards" logic:

1. is_compliant_activity (Global Sharia & Policy Filter):
   - Check the 'Type of Business' or 'Industry'.
   - Rule: Mark FALSE only if Alcohol, Gambling, Pork, or non-Islamic financial services are found.
   - Example: Automotive parts manufacturing is a clear PASS (TRUE).

2. license_status (Legal Standing):
   - GCC: Look for "Commercial Registration" (CR) or "Zakat" mentions.
   - International: Look for "Founded [X] years ago", "Incorporation", or "Business History".
   - Logic: If a company has been active for a long period (e.g., 30 years) and lists multiple locations, mark as "Valid" even if the physical license image isn't in the chunk.

3. tax_status (Fiscal Governance):
   - Scan for "Covenants", "Reporting", or "Obligations" sections.
   - Logic: If you find "All taxes must be filed" or "Tax returns required", mark as "Compliant". This indicates a structured tax compliance environment.

4. labor_compliance (Social & Regulatory):
   - GCC: Look for "Nitaqat" or "WPS".
   - International: Look for "Employees" or "Staff" counts.
   - Logic: A mention of a significant workforce (e.g., "100+ employees") indicates compliance with local labor and payroll standards. Mark "Compliant".

==================================================
STEP 3: OUTPUT FORMATTING (STRICT JSON)
==================================================
Return ONLY a valid JSON object matching `GCCComplianceOutput`.
- Fill ALL fields. Use 'Unknown' only if a deep scan of all chunks yields zero evidence.
- In `recommendation_notes`, justify your findings using text evidence (e.g., "Tax compliance confirmed via reporting covenants section").
- In `missing_documents`, list items that are legally required for the detected region but absent.
"""
# ==========================================
# 4. CREATE AGENT
# ==========================================
print("🚀 Creating Compliance Agent...")
compliance_agent = AgentFactory.create(
    name="Smart Compliance AuditorV2",
    description="Audits regulatory compliance adapting to GCC or International standards.",
    instructions=COMPLIANCE_INSTRUCTIONS,
    tools=[loan_kb],
    llm=ModelFactory.get("669a63646eb56306647e1091"),
    expected_output=GCCComplianceOutput
)
# compliance_agent.deploy()
print(f"✅ Agent Created: {compliance_agent.id}")
COMPLIANCE_AGENT_ID = compliance_agent.id

✅ Tools loaded: Loan Applications KBV2
🚀 Creating Compliance Agent...


/usr/local/lib/python3.12/dist-packages/aixplain/factories/agent_factory/__init__.py:146: UserWarning: Deprecating 'llm_id', use `llm` to define the large language model in agents.
  warnings.warn(


✅ Agent Created: 6985b3b0ff9be055415d53c6


In [ ]:
import json
from IPython.display import display, Markdown

# ============================================================================
# 🧪 SMART COMPLIANCE AGENT TEST CELL
# ============================================================================

# 1. تحديد الملف (نفس الملف السابق)
TARGET_APP_ID = "APP-TEST-001"

# 2. صياغة الأمر (Prompt)
# نطلب منه بوضوح تحديد الموقع أولاً
test_prompt = f"""
Perform a full Regulatory & Compliance Audit for Application ID: {TARGET_APP_ID}.

Mandatory Steps:
1. **Determine Geography:** Search for the borrower's address in the KB first.
2. **Apply Adaptive Standards:**
   - If the address is in GCC, check for Nitaqat, Saudization, and Zakat.
   - If the address is International (e.g., USA), check for Business License validity, Tax Returns, and Good Standing.
3. **Check Activity:** Verify if the business involves prohibited activities (Alcohol, Gambling, etc.).
4. **Output:** Return a valid JSON matching the GCCComplianceOutput schema.
"""

print(f"🚀 Starting Smart Compliance Audit for {TARGET_APP_ID}...")

try:
    # 3. تشغيل العميل
    # نعطيه وقتاً كافياً (Iterations) للبحث عن العنوان ثم الوثائق
    response = compliance_agent.run(test_prompt, max_iterations=15)

    print("-" * 60)
    print("✅ AUDIT COMPLETE")
    print("-" * 60)

    # 4. استخراج وتنسيق النتيجة
    final_output = None
    if hasattr(response, 'data') and hasattr(response.data, 'output'):
        final_output = response.data.output
    elif hasattr(response, 'output'):
        final_output = response.output
    else:
        final_output = response

    # 5. عرض النتيجة
    if isinstance(final_output, (dict, list)):
        print(json.dumps(final_output, indent=2, ensure_ascii=False))
    elif hasattr(final_output, 'model_dump'):
        print(json.dumps(final_output.model_dump(), indent=2, ensure_ascii=False))
    else:
        try:
            parsed = json.loads(str(final_output))
            print(json.dumps(parsed, indent=2, ensure_ascii=False))
        except:
            print(str(final_output))

except Exception as e:
    print(f"❌ Error: {str(e)}")

🚀 Starting Smart Compliance Audit for APP-TEST-001...
🤖  Smart Compliance AuditorV2 | ⏳ Initialization
⚙️  Smart Compliance AuditorV2 | GPT-4o Mini | ⏳
🤖  Smart Compliance AuditorV2 | ⏳ Formatting Output
🤖  Smart Compliance AuditorV2 | ✓ Formatting Output
✅ Done | (22.5 s total | 6 API calls | $0.004938)
------------------------------------------------------------
✅ AUDIT COMPLETE
------------------------------------------------------------
{
  "is_compliant_activity": true,
  "license_status": "Valid",
  "tax_status": "Compliant",
  "labor_compliance": "Compliant",
  "operating_region": "International/USA",
  "recommendation_notes": "Business is compliant with tax obligations as indicated by the reporting covenants section. The business has a valid license and has been operational for over 30 years, indicating good standing. No prohibited activities were found in the business operations.",
  "missing_documents": []
}


#TEAM AGENT

In [ ]:
try:
  LOAN_KB_ID = "6985a848037340c6889e1c51"
  print("🚀 got Loan Kb")
  collateral_agent = AgentFactory.get("6985b275ae1159fcbe5bbebb")
  print("🚀 got collateral agent...")
  financial_agent = AgentFactory.get("6985a902ff9be055415d4f98")
  print("🚀 got financial agent ...")
  industry_risk_agent = AgentFactory.get("6985b147ff9be055415d526b")
  print("🚀 got industry risk agent...")
  business_management_agent =AgentFactory.get("6985b388ae1159fcbe5bbf43")
  print("🚀 got business management agent...")
  compliance_agent = AgentFactory.get("6985b3b0ff9be055415d53c6")
  print("🚀 got compliance agent...")
except Exception as e:
    print(f"General Exception: {str(e)}")

🚀 got Loan Kb
🚀 got collateral agent...
🚀 got financial agent ...
🚀 got industry risk agent...
🚀 got business management agent...
🚀 got compliance agent...


In [ ]:


ORCHESTRATOR_INSTRUCTIONS = """
###  CRITICAL OPERATING PROTOCOL: SEQUENTIAL INVESTIGATION
You are the Lead Credit Officer. You are prohibited from issuing any final report until you complete a 5-step sequential investigation. Do not skip steps.

---

### STEP-BY-STEP FLOW (EXECUTE IN ORDER):

**PHASE 1: FINANCIALS**
1. Call `Financial Analysis AgentV3`.
2. Wait for the report. If data is found, proceed to Phase 2.

**PHASE 2: INDUSTRY**
1. Call `Industry PESTEL Analysis AgentV2`.
2. If the agent fails or returns an error, mark as "Technical Issue - KB Search Failed" but continue to Phase 3.

**PHASE 3: COLLATERAL**
1. Call `Collateral Analyst Agent`.
2. Instruction: "Search for inventory value and accounts receivable for [APP_ID]".
3. **MANDATORY**: You must attempt this call before moving to Phase 4.

**PHASE 4: MANAGEMENT**
1. Call `Business And Management Analysis Agent`.
2. Instruction: "Evaluate the management team and SWOT for [APP_ID]".
3. Wait for the report. Proceed to Phase 5.

"PHASE 5: COMPLIANCE AUDIT
1. Call `Smart Compliance AuditorV2`.
2. USE THIS EXACT PROMPT:
   'Perform a full Regulatory & Compliance Audit for [APP_ID].
    MANDATORY:
    1. Determine Geography: Search for the borrower address in the KB FIRST.
    2. Identify if USA or GCC.
    3. Apply adaptive standards and return the JSON.'
3. Wait for the response."

---

###  FINAL VALIDATION GATEWAY:
Before generating the final output, confirm you attempted all 5 calls:
-  Financials: [Status]
-  Industry: [Status or "Technical Issue"]
-  Collateral: [Status]
-  Management: [Status]
-  Compliance: [Status or "Technical Issue"]

**IF A TOOL FAILS (returns error):** You are allowed to mark it as "Technical Failure - Data Not Retrieved" and continue.
**IF A TOOL WAS NOT CALLED:** You MUST call it before proceeding.

---
When generating the final Credit Memo, you MUST EXPAND each section using the FULL content returned by the agents.

Do NOT produce short summaries.
Your role is NOT to summarize agents.
Your role is to:
• Synthesize
• Cross-validate
• Cite sources (agent lineage)
• Produce an audit-ready recommendation

You MUST explicitly reference which agent produced each insight.

Use phrases such as:
- "According to the Financial Analysis Agent…"
- "Based on the Industry PESTEL Agent findings…"
- "The Compliance Auditor indicates…"

This lineage is MANDATORY.

---

## DECISION TERMINOLOGY (CRITICAL GOVERNANCE RULE)

You are FORBIDDEN from issuing final approval.

Instead of:
Decision: APPROVE / DECLINE

You MUST write:
Credit Recommendation: APPROVE / REVIEW / DECLINE
**(Subject to Relationship Manager Validation)**

---

## SECTION EXPANSION + ACCEPTANCE CRITERIA ALIGNMENT

### 1) FINANCIAL ANALYSIS (ARD ENHANCED)

Using the Financial Analysis Agent output, you MUST include:

Liquidity Assessment:
• Ratios and working capital interpretation
• Explicit statement: Strong / Adequate / Weak liquidity

Profitability Assessment:
• Margins, ROE/ROA if available
• Trend commentary

Leverage Assessment:
• Debt indicators or absence of leverage data

MANDATORY ADDITIONS:
You must explicitly state:
• Liquidity Risk Level (Low/Moderate/High)
• Leverage Risk Level (Low/Moderate/High)
• Overall Financial Risk Level

If ratios are missing → explain risk of missing ratios.

This satisfies ARD financial scoring acceptance criteria.

---

### 2) INDUSTRY ANALYSIS (WITH LINEAGE)

Must begin with:
"According to the Industry PESTEL Analysis Agent…"

You must:
• Connect industry risks to borrower revenue and margins
• Distinguish structural vs cyclical vs emerging risks
• State Industry Risk Level (Low/Moderate/High)

---

### 3) COLLATERAL ANALYSIS (WITH LINEAGE)

Must begin with:
"According to the Collateral Analyst Agent…"

You must clearly state:
• Collateral Quality (Strong/Adequate/Weak)
• Collateral Liquidity (High/Medium/Low)
• Whether collateral strengthens credit decision

---

### 4) MANAGEMENT & BUSINESS ANALYSIS (WITH LINEAGE)

Must begin with:
"According to the Business & Management Analysis Agent…"

You must assign:
• Management Risk Level (Low/Moderate/High)
• Concentration Risk Level (Low/Moderate/High)

---

### 5) COMPLIANCE & REGULATORY ANALYSIS (CRITICAL ADDITION)

Must begin with:
"According to the Smart Compliance Auditor…"

You MUST explicitly state:
• Whether borrower operates in GCC or NON-GCC geography
• That regulatory standards were ADAPTED accordingly
• Compliance Risk Level (Low/Moderate/High)

If compliance incomplete → state regulatory uncertainty risk.

---

## NEW MANDATORY SECTION — RISK SCORECARD

Before the decision include:

Risk Summary:
• Financial Risk:
• Industry Risk:
• Collateral Strength:
• Management Risk:
• Compliance Risk:

This is REQUIRED for audit readiness.

---

## INTEGRATED RISK VIEW (KEEP)

Must connect cross-agent findings into a single narrative.

---

## GAP REPORT (UPGRADED)

For each missing item specify:
• Missing Data
• Impact on Credit Decision
• Responsible Agent / Tool

---

## DECISION RATIONALE (FINAL STEP)

Explain WHY the recommendation logically follows from:
Financial + Industry + Collateral + Management + Compliance.

---

## FINAL OUTPUT STRUCTURE

1) Credit Recommendation (Subject to RM Validation)
2) Financial Analysis
3) Industry Analysis
4) Collateral Analysis
5) Management & Business Analysis
6) Compliance Analysis
7) Risk Scorecard
8) Integrated Risk View
9) Gap Report
10) Decision Rationale

---

### WRITING STYLE
Executive credit memo.
Analytical, evidence-based, audit-ready.
Never produce short summaries.
"""
from aixplain.factories import TeamAgentFactory, ModelFactory
from aixplain.modules.agent.output_format import OutputFormat

# تجميع الفريق الموجود في الذاكرة
sub_agents = [
    financial_agent,
    industry_risk_agent,
    collateral_agent,
    business_management_agent,
    compliance_agent,
    # ingestion_tool
]

# أداة الرفع الخاصة بالمايسترو
# orchestrator_tools = [ingestion_tool]

print("🚀 Building the Lending Team...")

try:
    lending_assistant = TeamAgentFactory.create(
        name="AI Lending Assistant",
        description="Orchestrates full credit analysis. Ingests files, runs parallel agent analysis, and issues final credit memos.",
        agents=sub_agents,
        supervisor_llm=ModelFactory.get("6646261c6eb563165658bbb1"),
        # tools=orchestrator_tools,
        instructions=ORCHESTRATOR_INSTRUCTIONS,
        llm=ModelFactory.get("669a63646eb56306647e1091"), # GPT-4o Mini
        # output_format=OutputFormat.JSON,
        # expected_output=CreditMemoOutput

    )
    print(f"✅ Team Agent Ready: {lending_assistant.id}")

except Exception as e:
    print(f"❌ Error: {e}")

🚀 Building the Lending Team...
✅ Team Agent Ready: 698594bcae1159fcbe5bbad3


In [ ]:
response = lending_assistant.run(query="analysis APP-TEST-001 ", max_iterations=30,timeout=600)
print(response.data.output)

🤖  AI Lending Assistant | ⏳ Starting agent execution...
⚙️  Mentalist | GPT-4o Mini | ⏳ Agent 'Mentalist' is making LLM call: GPT-4o Mini
⚙️  Financial Analysis AgentV3 | GPT-4o Mini | ⏳ Agent 'Financial Analysis AgentV3' is making LLM call: GPT-4o Mini
⚙️  Financial Analysis AgentV3 | Loan Applications KBV2 | ⏳ Agent 'Financial Analysis AgentV3' is executing tool: Loan Applications KBV2
⚙️  Financial Analysis AgentV3 | GPT-4o Mini | ⏳ Agent 'Financial Analysis AgentV3' is making LLM call: GPT-4o Mini
⚙️  Financial Analysis AgentV3 | cal_financial_ratios | ✓ The output of the connection is:
SUCCESS - {'status': 'Success', 'data': "{'status': 'success',\n 'ratios': {'current_ratio': 3.1,\n  'quick_ratio': None,\n  'debt_to_equity': None,\n  'roe': 27.1,\n 
⚙️  Financial Analysis AgentV3 | GPT-4o Mini | ⏳ Agent 'Financial Analysis AgentV3' is making LLM call: GPT-4o Mini
⚙️  Smart Compliance AuditorV2 | GPT-4o Mini | ⏳ Agent 'Smart Compliance AuditorV2' is making LLM call: GPT-4o Mini
⚙️